In [ ]:
#Install the different library needed
!git clone https://gist.github.com/8409b3feec20f159d8a50b0a811d3bca.git
!pip install flwr
!pip install tensorflow

Cloning into '8409b3feec20f159d8a50b0a811d3bca'...
remote: Enumerating objects: 6, done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 6
Unpacking objects: 100% (6/6), done.
     |████████████████████████████████| 106 kB 15.2 MB/s 
     |████████████████████████████████| 4.1 MB 41.8 MB/s 
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.11.3
    Uninstalling importlib-metadata-4.11.3:
      Successfully uninstalled importlib-metadata-4.11.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.44.0
    Uninstalling grpcio-1.44.0:
      Successfully uninstalled grpcio-1.44.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10

In [ ]:
#Import the library
%run /content/8409b3feec20f159d8a50b0a811d3bca/draw.py

import tensorflow as tf
import flwr
import cv2
import numpy as np

In [ ]:
#Create the different folder where the training images will be stored
import os
!mkdir train

for i in range(10):
  os.system(" mkdir train/" + str(i))

In [ ]:
#Create some training example
draw(filename = "train/2/test1.jpg", w=28*15, h=28*15, line_width=15)
'''28*15'''

'28*15'

In [ ]:
#Create & compile the model
model = tf.keras.Sequential(
        [
            tf.keras.layers.Input(shape=(28, 28, 1), name = "input"),
            tf.keras.layers.Conv2D(32, (3, 3), activation="relu", name = "conv_1"),
            tf.keras.layers.MaxPooling2D((2, 2), name = "maxpool_1"),
            tf.keras.layers.Conv2D(64, (3, 3), activation="relu", name = "conv_2"),
            tf.keras.layers.MaxPooling2D((2, 2),name = "maxpool_2"),
            tf.keras.layers.Flatten(name = "flatten"),
            tf.keras.layers.Dense(64, activation="relu", name = "dense_1"),
            tf.keras.layers.Dense(10, activation="softmax", name = "output"),
        ]
    )

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)


In [ ]:
X_train = []
y_train = []

In [ ]:
for dirs in os.walk("train/", topdown=False):
  root, dirs1, files = dirs
  if files != []:
    for file in files:
      path = dirs[0] + "/"
      image = cv2.imread( path  + file,cv2.IMREAD_UNCHANGED)
      #make mask of where the transparent bits are
      trans_mask = image[:,:,3] == 0

      #replace areas of transparency with white and not transparent
      image[trans_mask] = [255, 255, 255, 255]

      #new image without alpha channel...
      new_img = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
      new_img = cv2.resize(new_img, dsize = (28,28), interpolation = cv2.INTER_CUBIC)
      gray = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)
      gray = gray
      for i in range(len(gray)):
        for j in range(len(gray[0])):
          if gray[i][j] == 255:
            gray[i][j] = 0
          else :
            gray[i][j] = 255

      cv2.imwrite("image_test1.jpg",gray)
      gray = gray/255
      gray = tf.reshape(gray,(1,)+gray.shape+(1,))
      X_train.append(gray)
      y_train.append(path[len(path)-2])

0
0
2
2
2


[<tf.Tensor: shape=(1, 28, 28, 1), dtype=float64, numpy=
 array([[[[0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
         